In [23]:
install.packages("dplyr")
install.packages("readxl")
install.packages("ggplot2")
library(ggplot2)
library(readxl)
library(dplyr)


customer_r <- read_excel("customer_r.xlsx")
reservation_r <- read_excel("reservation_r.xlsx")
order_info_r <- read_excel("order_info_r.xlsx")
item_r <- read_excel("item_r.xlsx")
colnames(customer_r) <- tolower(colnames(customer_r))
colnames(reservation_r) <- tolower(colnames(reservation_r))
colnames(order_info_r) <- tolower(colnames(order_info_r))
colnames(item_r) <- tolower(colnames(item_r))




Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [25]:
df_rsv_customer <- reservation_r %>%
select(customer_id, reserv_no) %>%    # 고객별 모든 예약 번호 선택
    arrange(customer_id, reserv_no)

head(df_rsv_customer)                 # 고객별 예약 번호 확인

customer_id,reserv_no
<chr>,<chr>
W1327595,2019061801
W1327595,2019071801
W1327803,2019060301
W1327803,2019091506
W1328432,2019060601
W1328432,2019060701


In [26]:
# (B) 스테이크 주문 예약 번호 데이터셋 생성
    df_steak_order_rsv_no <- order_info_r %>%
    filter(item_id == "M0005") %>%   # 스테이크 주문이면
    mutate(steak_order = "Y") %>%    # steak_order 열 데이터를 ‘Y’로 만듦
    arrange(reserv_no)

head(df_steak_order_rsv_no)          # 데이터셋 확인

order_no,item_id,reserv_no,quantity,sales,steak_order
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
190603578341100100,M0005,2019060301,4,140000,Y
190606607189610010,M0005,2019060601,4,140000,Y
190616137200510010,M0005,2019061601,3,105000,Y
190708517202310010,M0005,2019070801,2,70000,Y
190710578348100100,M0005,2019071002,3,105000,Y
190718107186310010,M0005,2019071801,4,140000,Y


In [27]:
# 고객의 모든 예약 번호(A)에 대해 스테이크 주문한 예약 번호(B)를 레프트 조인
df_steak_order_1 <- left_join(df_rsv_customer, df_steak_order_rsv_no, by = "reserv_no") %>%
    group_by(customer_id) %>%                                       # 고객 번호로 그룹화하여(182명)
    mutate(steak_order = ifelse(is.na(steak_order), "N", "Y")) %>%  # 주문 여부가 NA이면 N, Y이면 Y로 바꿈
    summarise(steak_order = max (steak_order)) %>%                  # 최댓값만 취함
    arrange(customer_id)

# 최종 정리된 고객별 스테이크 주문 여부
df_dpd_var <- df_steak_order_1

# 종속 변수, 최종 고객 182명의 스테이크 주문 여부 결과 확인
df_dpd_var

customer_id,steak_order
<chr>,<chr>
W1327595,Y
W1327803,Y
W1328432,Y
W1328505,N
W1328786,N
W1328985,Y
W1328993,Y
W1329560,N
W1329954,N


In [28]:
# 결측치 제거
df_customer <- customer_r %>% filter(!is.na(sex_code))
# 성별이 없으면(NA) 고객 번호 제거

# 고객 테이블과 예약 테이블 customer_id를 키로 이너 조인
df_table_join_1 <- inner_join(df_customer, reservation_r, by = "customer_id")

# df_table_join_1과 주문 테이블의 reserv_no를 키로 이너 조인
df_table_join_2 <- inner_join(df_table_join_1, order_info_r, by = "reserv_no")
str(df_table_join_2)     # df_table_join_2 테이블 구조 확인

tibble [322 × 19] (S3: tbl_df/tbl/data.frame)
 $ customer_id   : chr [1:322] "W1346506" "W1347648" "W1347756" "W1347984" ...
 $ customer_name : chr [1:322] "고객71" "고객72" "고객73" "고객74" ...
 $ phone_number  : chr [1:322] "010-1111-1181" "010-1111-1182" "010-1111-1183" "010-1111-1184" ...
 $ email         : chr [1:322] "scust71@sfnb.co.kr" "scust72@sfnb.co.kr" "scust73@sfnb.co.kr" "scust74@sfnb.co.kr" ...
 $ first_reg_date: chr [1:322] "19/09/01" "19/09/04" "19/09/05" "19/09/06" ...
 $ sex_code      : chr [1:322] "F" "M" "M" "M" ...
 $ birth         : chr [1:322] "19820922" "19940812" "19931217" "19810822" ...
 $ job           : chr [1:322] "자영업" "학생" NA "자영업" ...
 $ zip_code      : chr [1:322] "122100" "140100" "152100" "130100" ...
 $ reserv_no     : chr [1:322] "2019102002" "2019092301" "2019111301" "2019120201" ...
 $ reserv_date   : chr [1:322] "20191020" "20190923" "20191113" "20191202" ...
 $ reserv_time   : chr [1:322] "1800" "1800" "1200" "1200" ...
 $ branch        : chr [1:322]

In [29]:
# 고객 정보, 성별 정보와 방문 횟수, 방문객 수, 매출 합을 요약(코드 풀이에서 자세히 설명)
df_table_join_3 <- df_table_join_2 %>%
    group_by(customer_id, sex_code, reserv_no, visitor_cnt) %>% # ⓐ
    summarise(sales_sum = sum(sales)) %>%
    group_by(customer_id, sex_code) %>%                         # ⓑ
    summarise(visit_sum = n_distinct(reserv_no), visitor_sum = sum(visitor_cnt), sales_sum = sum(sales_sum) / 1000) %>%     # ⓒ
    arrange(customer_id)

df_idp_var <- df_table_join_3   # 독립 변수

df_idp_var                      # 독립 변수 확인(142행)

`summarise()` has grouped output by 'customer_id', 'sex_code', 'reserv_no'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'customer_id'. You can override using the `.groups` argument.



customer_id,sex_code,visit_sum,visitor_sum,sales_sum
<chr>,<chr>,<int>,<dbl>,<dbl>
W1327595,M,2,6,188
W1327803,M,2,6,210
W1328432,M,3,12,246
W1328505,F,5,15,287
W1328786,M,1,1,10
W1328985,M,2,6,178
W1328993,F,1,4,140
W1329560,M,1,2,24
W1329954,M,1,2,48


In [30]:
# 독립 변수 데이터셋(①-2)에 종속 변수 데이터셋(①-1) 이너 조인
df_final_data <- inner_join(df_idp_var, df_dpd_var, by = "customer_id")

# 의사 결정 나무 함수를 사용하려고 열 구조를 팩터형으로 바꿈
df_final_data$sex_code <- as.factor(df_final_data$sex_code)
df_final_data$steak_order <- as.factor(df_final_data$steak_order)

df_final_data <- df_final_data[, c(2:6)]   # 의사 결정 나무에 필요한 열만 선택
df_final_data                              # 최종 분석용 데이터셋 확인
 

sex_code,visit_sum,visitor_sum,sales_sum,steak_order
<fct>,<int>,<dbl>,<dbl>,<fct>
M,2,6,188,Y
M,2,6,210,Y
M,3,12,246,Y
F,5,15,287,N
M,1,1,10,N
M,2,6,178,Y
F,1,4,140,Y
M,1,2,24,N
M,1,2,48,N


In [31]:
install.packages("rpart")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [32]:
library(rpart)

In [33]:
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [34]:
library(caret)

In [35]:
install.packages("e1071")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [36]:
library(e1071)

In [37]:
set.seed(10000)

In [38]:
train_data <- createDataPartition(y = df_final_data$steak_order, p = 0.8, list = FALSE)
train <- df_final_data[train_data, ]
test <- df_final_data[-train_data, ]

In [39]:
decision_tree <- rpart(steak_order~., data = train)

In [40]:
decision_tree


n= 115 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

 1) root 115 44 N (0.61739130 0.38260870)  
   2) sales_sum< 55.5 37  2 N (0.94594595 0.05405405) *
   3) sales_sum>=55.5 78 36 Y (0.46153846 0.53846154)  
     6) sales_sum< 173 45 18 N (0.60000000 0.40000000)  
      12) visitor_sum>=5.5 19  1 N (0.94736842 0.05263158) *
      13) visitor_sum< 5.5 26  9 Y (0.34615385 0.65384615) *
     7) sales_sum>=173 33  9 Y (0.27272727 0.72727273)  
      14) visit_sum< 1.5 9  4 N (0.55555556 0.44444444) *
      15) visit_sum>=1.5 24  4 Y (0.16666667 0.83333333) *